# DPLA API

>The Digital Public Library of America built and maintains an open API to encourage the independent development of applications, tools, and resources that make use of data contained in the DPLA platform in new and innovative ways, from anywhere, at any time. For inspiration, consider developing an application that visualizes metadata in compelling ways, or a recommendation engine that suggests similar cultural heritage content based on user preferences or criteria, or a lightweight front-end interface for mobile devices. Or pursue something entirely different! The possibilities are endless.
>
> <https://pro.dp.la/developers/philosophy>

>The DPLA API is built following the [DPLA Metadata Application Profile (MAP)](https://pro.dp.la/hubs/metadata-application-profile), our internal metadata application profile. Currently, the DPLA API implements DPLA MAP version 3.1. The DPLA MAP is designed to build on the experience of the [Europeana Data Model](https://pro.europeana.eu/page/edm-documentation) (EDM) and to crosswalk with commonly used metadata standards, including [Dublin Core](http://dublincore.org/) (qualified and unqualified), [MODS](http://www.loc.gov/standards/mods/), and others. It references Dublin Core Terms, the Dublin Core Metadata Element Set, [ORE](http://www.openarchives.org/ore/), and a variety of controlled vocabularies and thesauri. The MAP will continue to evolve to implement more linked-data friendly methodologies to better share, expose, and connect DPLA’s content with users across the web.
>
> <https://pro.dp.la/developers/field-reference>

## `items` and `collections`

The top-level search containers are `items` and `collections`:

>A DPLA item is a reference to the digital representation of a single piece of content indexed by the DPLA. The piece of content can be, for example, a book, a photograph, a video, etc. The content is digitized from its original, physical source and uploaded to an online repository.
>
>…
>
>A `collection` is a little more abstract than an item. Where an item is a reference to the digital representation of a physical object, a `collection` is a representation of the grouping of a set of items.
>
> <https://pro.dp.la/developers/requests>


## the API key

The DPLA API key [is obtained](https://pro.dp.la/developers/policies#get-a-key) via automation, based on email address:

```bash
curl -v -XPOST https://api.dp.la/v2/api_key/YOUR_EMAIL@example.com
```

The convention in my studio is to store the key in a conventional JSON file found by a conventional environment variable:

In [1]:
import os
import json

from urllib import parse
from urllib import request

json_path = os.environ.get('SONGHAY_APP_SETTINGS_PATH')
json_file = open(json_path, 'r', encoding='utf-8')
app_settings = json.load(json_file)

dpla_set = app_settings['ProgramMetadata']['RestApiMetadataSet']['Dpla']

api_base, api_key = dpla_set['ApiBase'], dpla_set['ApiKey']

api_base

'https://api.dp.la/v2'

In [2]:
def get_json_string(search_container, query_string_params):

    query_string_params['api_key'] = api_key

    url = f'{api_base}/{search_container}?{parse.urlencode(query_string_params)}'

    response = request.urlopen(url)

    json_default = { 'error': None }

    json_string = json.dumps(json_default)

    if response.code != 200:
        json_default['error'] = { 'code': response.code }
        json_string = json.dumps(json_default)
    else:
        json_string = response.read().decode('UTF-8')

    return json_string


## `items` search by exact `sourceResource.title`

In [3]:
query_string_params = {
    'exact_field_match': "true",
    'sourceResource.title': 'the miseducation of the negro',
}

json_string = get_json_string('items', query_string_params)

json_d = json.loads(json_string)

with open('./dpla-exact-title-search.json', 'w', encoding='utf-8') as json_file:
    json.dump(json_d,
        json_file,
        ensure_ascii=False,
        indent=4,
        sort_keys=True)

In [4]:
%cat ./dpla-exact-title-search.json

{
    "count": 2,
    "docs": [
        {
            "@context": "http://dp.la/api/items/context",
            "@id": "http://dp.la/api/items/047f1c45f0ec0acbc12f54e86ae99748",
            "@type": "ore:Aggregation",
            "admin": {
                "object_status": 1
            },
            "aggregatedCHO": "#sourceResource",
            "dataProvider": "General Research Division. The New York Public Library",
            "id": "047f1c45f0ec0acbc12f54e86ae99748",
            "ingestDate": "2018-10-05T00:40:17.032805Z",
            "ingestType": "item",
            "isShownAt": "http://digitalcollections.nypl.org/items/510d47db-d501-a3d9-e040-e00a18064a99",
            "object": "http://images.nypl.org/index.php?id=495240&t=t",
            "originalRecord": {
                "collection": {
                    "@id": "http://dp.la/api/collections/267f1bbc5ed3a20ce9e448c6e2d206a6",
                    "id": "267f1bbc5ed3a20ce9e448c6e2d206a6"
                },
                

@[BryanWilhite](https://twitter.com/BryanWilhite)